In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle

In [ ]:
df = pd.read_csv('../data/dados_sinteticos_trem.csv', encoding='ISO-8859-1', sep='\t')
df

# Horario   TMP Dia da Semana  Lotacao  Atraso        Estacao  \
0        0   08:45  25.2       Segunda     39.7     5.6        Barueri   
1        1   00:15  14.3       Segunda     59.0     3.3    Barra Funda   
2        2   01:00  15.3       Segunda     76.2     5.6  Julio Prestes   
3        3   02:30  14.9         Terca     31.5     1.7  Julio Prestes   
4        4   01:00  10.3         Terca     66.4     0.2        Barueri   
...    ...     ...   ...           ...      ...     ...            ...   
4995  4995   02:15  10.3       Domingo     43.2     4.2         Osasco   
4996  4996   02:00  16.6       Segunda     59.0     4.2         Osasco   
4997  4997   07:30  12.2        Sabado     45.2     2.7         Osasco   
4998  4998   08:00  20.4        Sabado     77.3     1.0        Barueri   
4999  4999   11:45   8.4        Sabado     36.4     1.0        Barueri   

     Tipo de Trem  
0            Novo  
1            Novo  
2            Novo  
3            Novo  
4          Antigo  
...           ...  
4995         Novo  
4996       Antigo  
4997         Novo  
4998         Novo  
4999         Novo  

[5000 rows x 8 columns]

In [ ]:
def horario_para_minutos(hora):
    h, m = map(int, hora.split(":"))
    return h * 60 + m

df["Horario_min"] = df["Horario"].apply(horario_para_minutos)


X = df[["Horario_min", "Dia da Semana", "Lotacao", "Atraso"]]
y = df["TMP"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
cat_features = ["Dia da Semana"]
num_features = ["Horario_min", "Lotacao", "Atraso"]

preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
    ("num", "passthrough", num_features)
])

In [ ]:
from xgboost import XGBRegressor

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor(
        n_estimators=300,       
        max_depth=6,            
        learning_rate=0.1,      
        random_state=42,
        n_jobs=-1,              
        verbosity=1             
    ))
])
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Dia da Semana']),
                                                 ('num', 'passthrough',
                                                  ['Horario_min', 'Lotacao',
                                                   'Atraso'])])),
                ('regressor',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, early_stopping_rounds=None,
                              enabl...
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.1, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=6, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=300, n_jobs=-1, num_parallel_tree=1,
                              predictor='auto', random_state=42, reg_alpha=0,
                              reg_lambda=1, ...))])

In [ ]:
y_pred = pipeline.predict(X_test)


mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f'MAE: {mae:.2f}')
print(f'RMSE: {rmse:.2f}')

MAE: 2.64
RMSE: 3.14


In [ ]:
with open("mlp_previsao_trem.pkl", "wb") as f:
    pickle.dump(pipeline, f)